In [1]:
from transformers import pipeline, AutoModelForCausalLM, AutoTokenizer

# Load the model with offloading
model = AutoModelForCausalLM.from_pretrained(
    "defog/sqlcoder-7b-2",
    torch_dtype="float16",
    device_map="auto",  # Enable device mapping
    offload_folder="./offload"  # Specify the folder for offloading
)

# Load the tokenizer
tokenizer = AutoTokenizer.from_pretrained("defog/sqlcoder-7b-2")



ModuleNotFoundError: No module named 'transformers'

In [ ]:
# Create the pipeline
pipe = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    return_full_text=False 
)


In [ ]:
prompt = """
### Instructions:
Your task is to generate a SQL query based on the provided question and database schema. Follow these rules strictly:

1. **Read the question and schema thoroughly** to understand the required query.
2. **Use Table Aliases** to maintain clarity and prevent ambiguity.
3. **Cast the numerator as FLOAT** when calculating ratios.
4. **Do not repeat any part of the input, including the question, schema, or instructions, in the output.**
5. Your output must only contain the SQL query, formatted cleanly without additional text or explanations.

### Input:
Question: `Which salespeople generated the most sales?`
Schema:
CREATE TABLE products (
  product_id INTEGER PRIMARY KEY, -- Unique ID for each product
  name VARCHAR(50), -- Name of the product
  price DECIMAL(10,2), -- Price of each unit of the product
  quantity INTEGER  -- Current quantity in stock
);

CREATE TABLE customers (
   customer_id INTEGER PRIMARY KEY, -- Unique ID for each customer
   name VARCHAR(50), -- Name of the customer
   address VARCHAR(100) -- Mailing address of the customer
);

CREATE TABLE salespeople (
  salesperson_id INTEGER PRIMARY KEY, -- Unique ID for each salesperson
  name VARCHAR(50), -- Name of the salesperson
  region VARCHAR(50) -- Geographic sales region
);

CREATE TABLE sales (
  sale_id INTEGER PRIMARY KEY, -- Unique ID for each sale
  product_id INTEGER, -- ID of product sold
  customer_id INTEGER,  -- ID of customer who made purchase
  salesperson_id INTEGER, -- ID of salesperson who made the sale
  sale_date DATE, -- Date the sale occurred
  quantity INTEGER -- Quantity of product sold
);

CREATE TABLE product_suppliers (
  supplier_id INTEGER PRIMARY KEY, -- Unique ID for each supplier
  product_id INTEGER, -- Product ID supplied
  supply_price DECIMAL(10,2) -- Unit price charged by supplier
);

### Output:
"""

In [ ]:
# Provide the input prompt
# result = pipe(prompt, max_length=1000, num_return_sequences=1)

result = pipe(
    prompt,
    max_length=1000,
    num_return_sequences=1,
    eos_token_id=tokenizer.eos_token_id,  # Ensure clean output termination
    pad_token_id=tokenizer.pad_token_id,  # Avoid padding issues
    no_repeat_ngram_size=2,  # Avoid repetitive phrases
)

# Print the generated output
print(result[0]["generated_text"])